In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler ,RobustScaler
from sklearn.metrics import mean_squared_error,r2_score
import random
import yfinance as yf

In [17]:
start = "2017-01-01"
end = "2025-04-17" #04

In [19]:
SPY1 = yf.download("^GSPC", start=start, end=end)[['Close', 'Volume','High','Low']]
  

SPY1.columns = ['SPY_Close', 'SPY_Volume', 'SPY_High', 'SPY_Low']
print(SPY1.tail())

C:\Users\OMER\AppData\Local\Temp\ipykernel_33052\221333963.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  SPY1 = yf.download("^GSPC", start=start, end=end)[['Close', 'Volume','High','Low']]
[*********************100%***********************]  1 of 1 completed

              SPY_Close  SPY_Volume     SPY_High      SPY_Low
Date                                                         
2025-04-10  5268.049805  6677140000  5353.149902  5115.270020
2025-04-11  5363.359863  5602550000  5381.459961  5220.770020
2025-04-14  5405.970215  5031440000  5459.459961  5358.020020
2025-04-15  5396.629883  4317110000  5450.410156  5386.439941
2025-04-16  5275.700195  4607750000  5367.240234  5220.790039


**Creat false break/breach indicator**

In [23]:
SPY1['LowFalse'] = (
    (SPY1['SPY_Low'] < SPY1['SPY_Low'].shift(1)) &
    (SPY1['SPY_Close'] > SPY1['SPY_Low'].shift(1))
).astype(int)

SPY1['HighFalse'] = ((SPY1['SPY_High']> SPY1['SPY_High'].shift(1)) & (SPY1['SPY_Close'] < SPY1['SPY_High'].shift(1))).astype(int)

#Now on weekly basis

# Step 1: Assign a 'week' label to each row
SPY1['Week'] = SPY1.index.to_period('W')

# Step 2: Aggregate weekly data (low and close)
weekly_data = SPY1.groupby('Week').agg(
    Weekly_Low=('SPY_Low', 'min'),
    Weekly_Close=('SPY_Close', 'last')
)

# Step 3: Shift to get T−1 and T−2 values
weekly_data['Prev_1_Low'] = weekly_data['Weekly_Low'].shift(1)
weekly_data['Prev_2_Low'] = weekly_data['Weekly_Low'].shift(2)
weekly_data['Prev_1_Close'] = weekly_data['Weekly_Close'].shift(1)

# Step 4: Compute the condition for week T
weekly_data['LowBreakUp_Weekly'] = (
    (weekly_data['Prev_1_Low'] < weekly_data['Prev_2_Low']) &
    (weekly_data['Prev_1_Close'] > weekly_data['Prev_2_Low'])
).astype(int)

# Step 5: Map the weekly result back to the original daily DataFrame
SPY1['LowBreakUp_Weekly'] = SPY1['Week'].map(weekly_data['LowBreakUp_Weekly'])


# Now for high:


# Step 1: (If not already done) Label each row with its trading week
SPY1['Week'] = SPY1.index.to_period('W')

# Step 2: Aggregate weekly highs and closes
weekly_data = SPY1.groupby('Week').agg(
    Weekly_High=('SPY_High', 'max'),
    Weekly_Close=('SPY_Close', 'last')
)

# Step 3: Create shifted values for T−1 and T−2
weekly_data['Prev_1_High'] = weekly_data['Weekly_High'].shift(1)
weekly_data['Prev_2_High'] = weekly_data['Weekly_High'].shift(2)
weekly_data['Prev_1_Close'] = weekly_data['Weekly_Close'].shift(1)

# Step 4: Calculate the condition
weekly_data['HighBreakDown_Weekly'] = (
    (weekly_data['Prev_1_High'] > weekly_data['Prev_2_High']) &
    (weekly_data['Prev_1_Close'] < weekly_data['Prev_2_High'])
).astype(int)

# Step 5: Map the result back to the daily DataFrame
SPY1['HighBreakDown_Weekly'] = SPY1['Week'].map(weekly_data['HighBreakDown_Weekly'])


In [30]:
SPY1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2084 entries, 2017-01-03 to 2025-04-16
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype        
---  ------                --------------  -----        
 0   SPY_Close             2084 non-null   float64      
 1   SPY_Volume            2084 non-null   int64        
 2   SPY_High              2084 non-null   float64      
 3   SPY_Low               2084 non-null   float64      
 4   LowFalse              2084 non-null   int32        
 5   HighFalse             2084 non-null   int32        
 6   Week                  2084 non-null   period[W-SUN]
 7   LowBreakUp_Weekly     2084 non-null   int32        
 8   HighBreakDown_Weekly  2084 non-null   int32        
dtypes: float64(3), int32(4), int64(1), period[W-SUN](1)
memory usage: 130.2 KB


In [31]:
directory = 'raw_files'

import sys
import numpy.core.numeric as new_numeric

# Create a fake module under the old name
sys.modules['numpy._core.numeric'] = new_numeric

SPY = pd.read_pickle(r'raw_files\combined_cleaned_add_with_QQQ.pkl')
SPY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2055 entries, 0 to 2054
Data columns (total 46 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ds                   2055 non-null   datetime64[ns]
 1   SPY_Close            2055 non-null   float64       
 2   SPY_Volume           2055 non-null   int64         
 3   AAPL                 2055 non-null   float64       
 4   MSFT                 2055 non-null   float64       
 5   GOOG                 2055 non-null   float64       
 6   GLD                  2055 non-null   float64       
 7   SLV                  2055 non-null   float64       
 8   ^TNX                 2055 non-null   float64       
 9   DX-Y.NYB             2055 non-null   float64       
 10  JPY=X                2055 non-null   float64       
 11  EUR=X                2055 non-null   float64       
 12  USO                  2055 non-null   float64       
 13  UNG                  2055 non-nul

# Add support and resistence areas:

In [32]:
from scipy.signal import argrelextrema
spy_zones = SPY.copy()

# Use N = window size to define "local"
N = 10

# Find local maxima (resistance) and minima (support)
spy_zones['local_max'] = spy_zones['SPY_Close'].iloc[argrelextrema(spy_zones['SPY_Close'].values, np.greater_equal, order=N)[0]]
spy_zones['local_min'] = spy_zones['SPY_Close'].iloc[argrelextrema(spy_zones['SPY_Close'].values, np.less_equal, order=N)[0]]


# Define zone precision (e.g., $100 buckets)
zone_precision = 100
support_levels = spy_zones['local_min'].dropna().round(-int(np.log10(zone_precision)))
resistance_levels = spy_zones['local_max'].dropna().round(-int(np.log10(zone_precision)))

support_zones = support_levels.value_counts().loc[lambda x: x > 1].index.tolist()
resistance_zones = resistance_levels.value_counts().loc[lambda x: x > 1].index.tolist()

In [33]:
# define your margins
margin_sup = 50.0    # dollars above support to trigger 1
margin_res = 50.0    # dollars below resistance to trigger -1

def support_signal(price):
    # compute (distance, support) for only those supports below or equal to price
    diffs = [(price - s, s) for s in support_zones if s <= price]
    if not diffs:
        return 0
    # pick the smallest positive distance
    nearest_diff, nearest_sup = min(diffs, key=lambda x: x[0])
    return 1 if nearest_diff <= margin_sup else 0

def resistance_signal(price):
    # compute (distance, resistance) for only those resistances above or equal to price
    diffs = [(r - price, r) for r in resistance_zones if r >= price]
    if not diffs:
        return 0
    nearest_diff, nearest_res = min(diffs, key=lambda x: x[0])
    return -1 if nearest_diff <= margin_res else 0

# apply to your DataFrame
spy_zones['SUPPORT_Signal']    = spy_zones['SPY_Close'].apply(support_signal)
spy_zones['RESISTANCE_Signal'] = spy_zones['SPY_Close'].apply(resistance_signal)

# quick check
print(spy_zones[['SPY_Close','SUPPORT_Signal','RESISTANCE_Signal']].tail(60))


        SPY_Close  SUPPORT_Signal  RESISTANCE_Signal
1995  6086.370117               0                 -1
1996  6118.709961               0                  0
1997  6101.240234               0                  0
1998  6012.279785               0                  0
1999  6067.700195               0                 -1
2000  6039.310059               0                  0
2001  6071.169922               0                 -1
2002  6040.529785               0                  0
2003  5994.569824               0                  0
2004  6037.879883               0                  0
2005  6061.479980               0                 -1
2006  6083.569824               0                 -1
2007  6025.990234               0                  0
2008  6066.439941               0                 -1
2009  6068.500000               0                 -1
2010  6051.970215               0                 -1
2011  6115.069824               0                  0
2012  6114.629883               0             

In [34]:
spy_zones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2055 entries, 0 to 2054
Data columns (total 50 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ds                   2055 non-null   datetime64[ns]
 1   SPY_Close            2055 non-null   float64       
 2   SPY_Volume           2055 non-null   int64         
 3   AAPL                 2055 non-null   float64       
 4   MSFT                 2055 non-null   float64       
 5   GOOG                 2055 non-null   float64       
 6   GLD                  2055 non-null   float64       
 7   SLV                  2055 non-null   float64       
 8   ^TNX                 2055 non-null   float64       
 9   DX-Y.NYB             2055 non-null   float64       
 10  JPY=X                2055 non-null   float64       
 11  EUR=X                2055 non-null   float64       
 12  USO                  2055 non-null   float64       
 13  UNG                  2055 non-nul

In [35]:
print(SPY.columns.to_list())

['ds', 'SPY_Close', 'SPY_Volume', 'AAPL', 'MSFT', 'GOOG', 'GLD', 'SLV', '^TNX', 'DX-Y.NYB', 'JPY=X', 'EUR=X', 'USO', 'UNG', 'BTC-USD', 'CPER', '^VIX', '^GDAXI', '^FTSE', '^RUT', '^N225', 'IEI', 'CNYUSD=X', '2Y_Yield', 'yield_curve', 'market_closed_count', 'yield_curve_term', 'high-low', 'before_high-low', 'SPY_RSI', 'RSI_rank', 'RSI_rank_2', 'EMA_20', 'EMA_50', 'EMA_200', 'EMA_20_50', 'EMA_50_200', 'EMA_50_diff', 'EMA_200_diff', 'SPY_std', 'SPY_mean', 'SPY_30', 'QQQ_Close', 'QQQ_Volume', 'qqq_std', 'qqq_mean']


In [36]:
SPY['EMA_20_50'].head(30)

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
Name: EMA_20_50, dtype: int64

### Add RSI & MACD 

In [39]:
def calculate_rsi(prices, period=14):
    prices = pd.to_numeric(prices, errors='coerce')
    delta  = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

  # MACD Calculation
def calculate_macd(prices, fast=12, slow=26):
    prices = pd.Series(prices).astype(float).dropna()
    exp1 = prices.ewm(span=fast, adjust=False).mean()
    exp2 = prices.ewm(span=slow, adjust=False).mean()
    return exp1 - exp2

# number of consecative up/down days     
def calculate_consecutive_streak(close_series: pd.Series) -> pd.Series:
    """
    Calculate the consecutive up/down streaks for a close price series.
    - If today's close > yesterday's, streak = previous_streak + 1 (or 1 if previous_streak <= 0)
    - If today's close < yesterday's, streak = previous_streak - 1 (or -1 if previous_streak >= 0)
    - If equal, streak = 0
    """
    streak = [0] * len(close_series)
    for i in range(1, len(close_series)):
        if close_series.iat[i] > close_series.iat[i - 1]:
            streak[i] = streak[i - 1] + 1 if streak[i - 1] > 0 else 1
        elif close_series.iat[i] < close_series.iat[i - 1]:
            streak[i] = streak[i - 1] - 1 if streak[i - 1] < 0 else -1
        else:
            streak[i] = 0
    return pd.Series(streak, index=close_series.index, name='consecutive_streak')

# Add some internal features 
#
def model_own_features(
    data: pd.DataFrame,
    column: str,
    keep_columns = None
) -> pd.DataFrame:
    """
    Add RSI, MACD, rolling means, std, and consecutive streak for `column`.
    Optionally preserve only keep_columns + new features.
    """
    df = data.copy()
    # Ensure numeric
    df[column] = pd.to_numeric(df[column], errors='coerce')
    
    # 1) RSI
    df['RSI_14'] = calculate_rsi(df[column], period=14)
    
    # 2) MACD
    #macd_df = calculate_macd(df[column])
    #df = df.join(macd_df)
    df['MACD'] = calculate_macd(df[column])
    # 3) Rolling stats
    #df['20d_mean'] = df[column].rolling(20).mean()
    df['5d_mean']  = df[column].rolling(5).mean()
    df['5d_std']   = df[column].rolling(5).std()
    
    # 4) Consecutive streak
    df['streak'] = calculate_consecutive_streak(df[column])
    
    # 5) Select output columns
    if keep_columns is not None:
        # always include the new features
        new_feats = ['RSI_14', 'MACD', '5d_mean', '5d_std', 'streak']
        df = df[ keep_columns + new_feats ]

    df = df.dropna()

    return df


In [40]:
# calculate the cosecative returns
def calculate_consecutive_return(close_series: pd.Series, streak_series: pd.Series) -> pd.Series:
    """
    Calculate the percentage change over the span of the consecutive streak.
    - If streak == 0: 0
    - Else: (close_today / close_n_days_ago - 1) * 100, where n = abs(streak)
    """
    returns = []
    prices = close_series.values
    streaks = streak_series.values.astype(int)
    for i, (price, streak) in enumerate(zip(prices, streaks)):
        n = abs(streak)
        if n > 0 and i - n >= 0:
            prev_price = prices[i - n]
            returns.append((price / prev_price - 1) * 100)
        else:
            returns.append(0.0)
    return pd.Series(returns, index=close_series.index, name='abs_consecutive_change')

def date_features(df):
    df['month'] = df['ds'].dt.month
    df['day_of_week'] = df['ds'].dt.dayofweek
    df['day_of_week'] = df['day_of_week'] + 2
    return df

In [43]:
#initiating the df for modelling
cols_to_keep = ['EMA_20', 'EMA_50', 'EMA_200', 'EMA_50_diff', 'EMA_200_diff','SPY_Volume','high-low','before_high-low','ds','SPY_Close']
result = model_own_features(
    data=SPY,
    column='SPY_Close',
    keep_columns= cols_to_keep
)

result['abs_consecutive_change'] = calculate_consecutive_return(result['SPY_Close'], result['streak'])
result = date_features(result) # to add date features
result = result.drop('ds',axis=1) # drop the date column
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2042 entries, 13 to 2054
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   EMA_20                  2042 non-null   float64
 1   EMA_50                  2042 non-null   float64
 2   EMA_200                 2042 non-null   float64
 3   EMA_50_diff             2042 non-null   float64
 4   EMA_200_diff            2042 non-null   float64
 5   SPY_Volume              2042 non-null   int64  
 6   high-low                2042 non-null   float64
 7   before_high-low         2042 non-null   float64
 8   SPY_Close               2042 non-null   float64
 9   RSI_14                  2042 non-null   float64
 10  MACD                    2042 non-null   float64
 11  5d_mean                 2042 non-null   float64
 12  5d_std                  2042 non-null   float64
 13  streak                  2042 non-null   int64  
 14  abs_consecutive_change  2042 non-null   floa

# Add the advance features :

In [45]:
#SPY1.info()
SPY1.reset_index(inplace = True)
SPY1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2084 entries, 0 to 2083
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  2084 non-null   datetime64[ns]
 1   SPY_Close             2084 non-null   float64       
 2   SPY_Volume            2084 non-null   int64         
 3   SPY_High              2084 non-null   float64       
 4   SPY_Low               2084 non-null   float64       
 5   LowFalse              2084 non-null   int32         
 6   HighFalse             2084 non-null   int32         
 7   Week                  2084 non-null   period[W-SUN] 
 8   LowBreakUp_Weekly     2084 non-null   int32         
 9   HighBreakDown_Weekly  2084 non-null   int32         
dtypes: datetime64[ns](1), float64(3), int32(4), int64(1), period[W-SUN](1)
memory usage: 130.4 KB


In [46]:
SPY1.rename(columns = {'Date': 'ds'}, inplace =True)
SPY1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2084 entries, 0 to 2083
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ds                    2084 non-null   datetime64[ns]
 1   SPY_Close             2084 non-null   float64       
 2   SPY_Volume            2084 non-null   int64         
 3   SPY_High              2084 non-null   float64       
 4   SPY_Low               2084 non-null   float64       
 5   LowFalse              2084 non-null   int32         
 6   HighFalse             2084 non-null   int32         
 7   Week                  2084 non-null   period[W-SUN] 
 8   LowBreakUp_Weekly     2084 non-null   int32         
 9   HighBreakDown_Weekly  2084 non-null   int32         
dtypes: datetime64[ns](1), float64(3), int32(4), int64(1), period[W-SUN](1)
memory usage: 130.4 KB


**Merge:**

In [50]:
result_final = result.merge(SPY1[['LowFalse','HighFalse','LowBreakUp_Weekly','HighBreakDown_Weekly','ds']], on='ds', how='Left')
result_final.info()

KeyError: 'ds'